In [3]:
import os
import sqlite3
from dask.distributed import Client, LocalCluster

# set up cluster and workers
cluster = LocalCluster(n_workers=4,
threads_per_worker=1,
memory_limit='8GB') #my RAM size
client = Client(cluster)

# have a look at your workers client
import modin.pandas as pd
import seaborn as sns
import numpy as np
import re
#this sets up dask which is a multi processor library for data frames, 
#prob best to change the 64GB to 16GB and
#if this works you could try increasing the number of workers
#SEE PROGRESS OF DASK RUNTIME FIND LIBRARY
#what codes do and why in notebook
#but in dissertation you explain in more detail

C:\Users\44795\anaconda3\lib\site-packages\distributed\node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 55897 instead
  warnings.warn(


In [4]:
def getlongestmatch(seq, umin = '2', umax = '', gapmin = '1', gapmax = '2', repeatmin = '2', repeatmax = ''):
  m = re.finditer('(([C]{' +umin+ ',' +umax+ '}[AGT]{' +gapmin+ ',' +gapmax+ '}){' +repeatmin+ ',' +repeatmax+ '})|(([G]{' +umin+ ',' +umax+ '}[ACT]{' +gapmin+ ',' +gapmax+ '}){' +repeatmin+ ',' +repeatmax+ '})',seq, flags = re.IGNORECASE)
  longest = ''
  lcount = 0
  span = ''
  for match in m:
    lcount += 1
    if len(match.group()) > len(longest):
      longest = match.group() #.group() tells you what exactly they found e.g. CCCCCC
      span = match.span() #tells you where exactly a match is found (startindx,endindex)
  m = re.finditer('(([C]{' +umin+ ',' +umax+ '}[AGT]{' +gapmin+ ',' +gapmax+ '}){' +repeatmin+ ',' +repeatmin+ '})|(([G]{' +umin+ ',' +umax+ '}[ACT]{' +gapmin+ ',' +gapmax+ '}){' +repeatmin+ ',' +repeatmin+ '})',seq, flags = re.IGNORECASE)
  count = 0
  for match in m:
    count += 1
  return longest, len(longest)#, span, lcount, count
#function to find the longest match of...GC?
#variable m stores function that iterates and finds through a genetic sequence string, from left to right I'm guessing C or G?
#m is short for match 
#print different things!!
#play around with regex and then plot different ones, some with Gs somewith Cs

In [5]:
def shortrepeat(seq, umin = '4', umax = '', gapmin = '1', gapmax = '', repeatmin = '1', repeatmax = ''):
    match = re.search('(([C]{' +umin+ ',' +umax+ '}[ACGT]{' +gapmin+ ',' +gapmax+ '}){' +repeatmin+ ',' +repeatmin+ '})|(([G]{' +umin+ ',' +umax+ '}[ACT]{' +gapmin+ ',' +gapmax+ '}){' +repeatmin+ ',' +repeatmin+ '})',seq, flags = re.IGNORECASE)
    if match is None:
        return ("")
    else:
        return match.group(0)
#    if match:
      #  return match.group(0)
    #else:
     #   return ("")

In [6]:
#to incorporate in the dataframe you make lambda function again

def kmers(tseq,kmer):
    tseq = tseq.lower()
    start = 0
    end = kmer
    gcs = {} #dictionary
    while (end <= len(tseq)):
        tar = tseq[start:end]
        if tar not in gcs:
            gcs[tar] = 0
        gcs[tar] += 1
        start += 1
        end += 1
    return (gcs)

def gcminmax(tseq,win):
  tseq = tseq.lower()
  acgt = True
  if ((tseq.count('a') + tseq.count('c') + tseq.count('g') + tseq.count('t')) != len(tseq)):
    acgt = False
  target = len(tseq)
  start = 0
  end = win
  gcs = []
  mingcs = 1
  maxgcs = 0
  while (end <= target):
    tar = tseq[start:end]
    new = (tar.count('g') + tar.count('c')) / len(tar)
    if new < mingcs:
      mingcs = new
    if new > maxgcs:
      maxgcs = new
    start += 1
    end += 1
  return (mingcs,maxgcs,acgt)

In [7]:
# this function copied from gquadfinder
def WriteSeq(line,liste, LISTE, F, Len ):
    i,k,I=0,0,0
    a=b=LISTE[i]
    MSCORE=[]
    if len(LISTE) > 1:
        c = LISTE[i+1]
        while i < len(LISTE)-2:
            if c == b+1:
                k = k+1
                i = i+1
            else:
                I = I+1
                seq = line[a:a+F+k]
                sequence, liste2 = BaseScore(seq)
                MSCORE.append(abs(round(np.mean(liste2), 2)))
                k = 0
                i = i+1
                a = LISTE[i]
            b = LISTE[i]
            c = LISTE[i+1]
        I=I+1
        seq=line[a:a+F+k+1]
        sequence, liste2 = BaseScore(seq)
        MSCORE.append(abs(round(np.mean(liste2), 2)))
    else:
        I = I+1
        seq = line[a:a+F]
        MSCORE.append(abs(liste[a]))
    return MSCORE
# this function copied from gquadfinder
def BaseScore(line):
    item, liste = 0, []
    while item < len(line):
        if item < len(line) and (line[item] == "G" or line[item] == "g"):
            liste.append(1)
            if item + 1 < len(line) and (line[item + 1] == "G" or line[item + 1] == "g"):
                liste[item] = 2
                liste.append(2)
                if item + 2 < len(line) and (line[item + 2] == "G" or line[item + 2] == "g"):
                    liste[item + 1] = 3
                    liste[item] = 3
                    liste.append(3)
                    if item + 3 < len(line) and (line[item + 3] == "G" or line[item + 3] == "g"):
                        liste[item] = 4
                        liste[item + 1] = 4
                        liste[item + 2] = 4
                        liste.append(4)
                        item = item + 1
                    item = item + 1
                item = item + 1
            item = item + 1
            while item < len(line) and (line[item] == "G" or line[item] == "g"):
                liste.append(4)
                item = item + 1
        elif item < len(line) and line[item] != "G" and line[item] != "g" and line[item] != "C" and line[item] != "c":
            liste.append(0)
            item = item + 1
        elif item < len(line) and (line[item] == "C" or line[item] == "c"):
            liste.append(-1)
            if item + 1 < len(line) and (line[item + 1] == "C" or line[item + 1] == "c"):
                liste[item] = -2
                liste.append(-2)
                if item + 2 < len(line) and (line[item + 2] == "C" or line[item + 2] == "c"):
                    liste[item + 1] = -3
                    liste[item] = -3
                    liste.append(-3)
                    if item + 3 < len(line) and (line[item + 3] == "C" or line[item + 3] == "c"):
                        liste[item] = -4
                        liste[item + 1] = -4
                        liste[item + 2] = -4
                        liste.append(-4)
                        item = item + 1
                    item = item + 1
                item = item + 1
            item = item + 1
            while item < len(line) and (line[item] == "C" or line[item] == "c"):
                liste.append(-4)
                item = item + 1
        else:
            # la fin du la ligne ou y a des entrers
            item = item + 1
    return line, liste
# this function copied from gquadfinder
def CalScore(liste, k):
    Score_Liste=[]
    for i in range(len(liste)-(k-1)):
        j, Sum = 0, 0
        while j < k:
            Sum = Sum + liste[i]
            j = j + 1
            i = i + 1
        Mean=Sum/float(k)
        Score_Liste.append(Mean)
    return Score_Liste
# this function copied from gquadfinder
def GetG4(line, liste, Window, k, Len):
    LG4 = []
    for i in range(len(liste)):
        if liste[i] >= float(Window) or liste[i] <= - float(Window):
            seq = line[i:i+k]
            LG4.append(i)
    return LG4

'''we use this function for the dataframe but we have to keep the other functions as they are used in this function'''
def gquadcheck(sequence):
    minscore = 2
    window = 50
    cseq, scores = BaseScore(sequence)
    score = CalScore(scores, window)
    outG4 = GetG4(sequence, score, minscore, window, len(scores))
    if len(outG4) == 0:
        return False, []
    mscore = WriteSeq(sequence, score, outG4, window, len(scores))
    if len(mscore):
        return True, mscore
    else:
        return False, mscore

In [13]:
gcminmax('GAATGGTGCCAGGGGCAGAGGGGGCAATGCCGGGG',10)

(0.5, 0.8, True)

In [14]:
kmers('GAATGGTGCCAGGGGCAGAGGGGGCAATGCCGGGGCCCAGGT',2)

{'ga': 2,
 'aa': 2,
 'at': 2,
 'tg': 3,
 'gg': 12,
 'gt': 2,
 'gc': 5,
 'cc': 4,
 'ca': 4,
 'ag': 4,
 'cg': 1}

In [9]:
coredata = pd.read_pickle('core.pickle')

To request implementation, send an email to feature_requests@modin.org.


In [188]:
coredata.to_pickle('core.pickle')

In [32]:
coredata
#pd.set_option('display.max_columns', None)
#coredata = coredata.reindex(columns=column_names)

,chromosome,start_idx,end_idx,sequence,state_name,gc,gc_min,gc_max,repchr,repstart,repend,gquadscore,gquadscoremax,longest,length,match.group(0),mingcs,maxgcs,acgt,gquadscoremin,gquadCount,gquadDensity per kb
1,chr1,10801,11600,ACATGCTAGCGCGTCGGGGTGGAGGCGTGGCGCAGGCGCAGAGAGG...,DELETION,63.0,30.0,75.0,chr1,10801.0,11600.0,[],0.00,GGGGGTTGGGGGGGC,15,GGGGT,0.30,0.80,True,0.0,0,0.000000
42,chr1,81001,81500,ACCAGGGAGTCGGAGGTGTTGCAGTGAGCCGAGATCGTGCCACTGC...,DELETION,40.0,32.0,53.0,chr1,81001.0,81500.0,[],0.00,,0,,0.31,0.54,True,0.0,0,0.000000
44,chr1,81901,82300,ATCTTGTTCTTTAAACAAATCAACAAAGTTGACAAAAAAATTAGAT...,DELETION,28.0,24.0,35.0,chr1,81901.0,82300.0,[],0.00,,0,,0.22,0.37,True,0.0,0,0.000000
163,chr1,488601,489100,TGGTCACACCCTGTCCTCCTCCTACACGTACTCGGATGCTTCCTCC...,DELETION,62.0,56.0,68.0,chr1,488601.0,489100.0,[],0.00,CCCCAACCCCTGGGG,15,CCCCCTACGG,0.57,0.71,True,0.0,0,0.000000
218,chr1,875901,876400,CCACGCTCGCCCACGCTCCTCCCCCACACTCTCCCACACTCCCCCA...,DELETION,69.0,66.0,71.0,chr1,875901.0,876400.0,"[1.94, 1.93, 1.98, 1.98, 1.98, 1.94, 2.03, 2.09]",2.09,,0,CCCCCACACT,0.65,0.72,True,0.0,8,16.032064
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74210,chr9,137551600,137551799,CAGTGGAGACGCCAGGGACCACCGCGCTGAGAGCGATCACTCATCC...,DELETION,85.0,80.0,90.0,chr9,137551600.0,137551799.0,"[1.98, 1.97, 1.97, 1.93]",1.98,CCCCGCCCCGCCCCG,15,CCCCGCCCC,0.80,0.93,True,0.0,4,20.100503
74216,chr9,137565700,137566899,GAGGAAGCTCCCCTGGGTGACTCTGTCTGTGAGGAAGCTCCCCTGG...,DELETION,61.0,59.0,64.0,chr9,137565700.0,137566899.0,[],0.00,,0,CCCCTGGGT,0.58,0.66,True,0.0,0,0.000000
74218,chr9,137569300,137569599,ACCCATGCCCCCACCCACCCCCCACCCACCTTCCATCCATCTACCC...,DELETION,57.0,52.0,62.0,chr9,137569300.0,137569599.0,[],0.00,,0,CCCCCACCCA,0.52,0.62,True,0.0,0,0.000000
74223,chr9,137590300,137590599,TCGGGAGGCGCCGAGCGGGGGCCGGGGCGAGGCCGCGGCGCGCCGG...,DELETION,85.0,83.0,87.0,chr9,137590300.0,137590599.0,[],0.00,GGGGGCCGGGGC,12,GGGGGCC,0.82,0.89,True,0.0,0,0.000000


In [137]:
pd.set_option('display.max_columns', None)

In [47]:
del coredata['end_idx']

In [179]:
coredata['longest'],coredata['length'] = zip(*coredata.sequence.apply(lambda x: getlongestmatch(x,'4','','1','5','2','')))
#longest runtime here-only have to execute once
#took like 5-10mins

Exception ignored in: <function Future.__del__ at 0x000001D78A716940>
Traceback (most recent call last):
  File "C:\Users\44795\anaconda3\lib\site-packages\distributed\client.py", line 380, in __del__
    self.release()
  File "C:\Users\44795\anaconda3\lib\site-packages\distributed\client.py", line 355, in release
    self.client.loop.add_callback(self.client._dec_ref, stringify(self.key))
  File "C:\Users\44795\anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 227, in add_callback
    call_soon(self._run_callback, functools.partial(callback, *args, **kwargs))
  File "C:\Users\44795\anaconda3\lib\asyncio\base_events.py", line 770, in call_soon_threadsafe
    self._write_to_self()
  File "C:\Users\44795\anaconda3\lib\asyncio\selector_events.py", line 140, in _write_to_self
    csock.send(b'\0')
KeyboardInterrupt: 


In [118]:
coredata['match.group(0)'] = coredata.sequence.apply(lambda x: shortrepeat(x,'4','','1','2','1',''))
#look at pickle instead of .copy() for this cell

In [31]:
coredata['mingcs'],coredata['maxgcs'],coredata['acgt'] = zip(*coredata.sequence.apply(lambda x: gcminmax(x, 100)))

In [38]:
coredata['gquadcheck'],coredata['gquadscore'] = zip(*coredata.sequence.apply(lambda x: gquadcheck(x)))

#counting the gquadscore array,np.amin, np.amean (look into it) then use to plot boxplot, 
#create boxplot on gquad score to compare each istate
#read more on gquadscore paper sent on teams

In [48]:
coredata

,chromosome,sequence,state_name,gc,gc_min,gc_max,repchr,repeat_seq,hmm_state_id,has_gquad,has_repeats,gquadscoremax,longest,length,match.group(0),mingcs,maxgcs,acgt,gquadscoremin,gquadCount,gquadDensity per kb,gquadscore
1,chr1,ACATGCTAGCGCGTCGGGGTGGAGGCGTGGCGCAGGCGCAGAGAGG...,DELETION,63.0,30.0,75.0,chr1,CATGCTAGCGCGTCCAGGGGTGGAGGCGTGGCGCAGGCGCAGAGAG...,3.0,0.0,1.0,0.00,GGGGGTTGGGGGGGC,15,GGGGT,0.30,0.80,True,0.0,0,0.000000,[]
42,chr1,ACCAGGGAGTCGGAGGTGTTGCAGTGAGCCGAGATCGTGCCACTGC...,DELETION,40.0,32.0,53.0,chr1,NO_REPEATS,3.0,0.0,0.0,0.00,,0,,0.31,0.54,True,0.0,0,0.000000,[]
44,chr1,ATCTTGTTCTTTAAACAAATCAACAAAGTTGACAAAAAAATTAGAT...,DELETION,28.0,24.0,35.0,chr1,NO_REPEATS,3.0,0.0,0.0,0.00,,0,,0.22,0.37,True,0.0,0,0.000000,[]
163,chr1,TGGTCACACCCTGTCCTCCTCCTACACGTACTCGGATGCTTCCTCC...,DELETION,62.0,56.0,68.0,chr1,NO_REPEATS,3.0,0.0,0.0,0.00,CCCCAACCCCTGGGG,15,CCCCCTACGG,0.57,0.71,True,0.0,0,0.000000,[]
218,chr1,CCACGCTCGCCCACGCTCCTCCCCCACACTCTCCCACACTCCCCCA...,DELETION,69.0,66.0,71.0,chr1,TCCCCCACAC,3.0,1.0,1.0,2.09,,0,CCCCCACACT,0.65,0.72,True,0.0,8,16.032064,"[1.94, 1.93, 1.98, 1.98, 1.98, 1.94, 2.03, 2.09]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74210,chr9,CAGTGGAGACGCCAGGGACCACCGCGCTGAGAGCGATCACTCATCC...,DELETION,85.0,80.0,90.0,chr9,CGCGAGCCCCGCCC,3.0,1.0,1.0,1.98,CCCCGCCCCGCCCCG,15,CCCCGCCCC,0.80,0.93,True,0.0,4,20.100503,"[1.98, 1.97, 1.97, 1.93]"
74216,chr9,GAGGAAGCTCCCCTGGGTGACTCTGTCTGTGAGGAAGCTCCCCTGG...,DELETION,61.0,59.0,64.0,chr9,GAGGAAGCTCCCCTGGGTGACTCTGTCTGT,3.0,0.0,1.0,0.00,,0,CCCCTGGGT,0.58,0.66,True,0.0,0,0.000000,[]
74218,chr9,ACCCATGCCCCCACCCACCCCCCACCCACCTTCCATCCATCTACCC...,DELETION,57.0,52.0,62.0,chr9,CCAT,3.0,0.0,1.0,0.00,,0,CCCCCACCCA,0.52,0.62,True,0.0,0,0.000000,[]
74223,chr9,TCGGGAGGCGCCGAGCGGGGGCCGGGGCGAGGCCGCGGCGCGCCGG...,DELETION,85.0,83.0,87.0,chr9,NO_REPEATS,3.0,0.0,0.0,0.00,GGGGGCCGGGGC,12,GGGGGCC,0.82,0.89,True,0.0,0,0.000000,[]


In [ ]:
coredata["hmm_state_id"].replace({: "x", "b": "y"}, inplace=True)
print(df)

In [13]:
coredata['gquadscoremax'] = coredata.gquadscore.apply(lambda x: np.amax(x, initial=0))

In [143]:
coredata['gquadscoremin'] = coredata.gquadscore.apply(lambda x: np.amin(x, initial=0))

In [170]:
coredata['gquadCount'] = coredata.gquadscore.apply(lambda x: len(x))

In [177]:
#to find density of gquad per row
coredata['gquadDensity per kb'] = coredata['gquadCount']/coredata['region_length'] * 1000

In [19]:
tcdata = pd.read_pickle('tufcore.pickle')

In [187]:
tcdata.to_pickle('tufcore.pickle')

In [71]:
tcdata

,chromosome,start_idx,end_idx,sequence,state_name,gc,gc_min,gc_max,repchr,repstart,repend,gquadscoremax,gquadscoremin,mingcs,maxgcs,acgt,longest,length,match.group(0),gquadCount,gquadDensity per kb,gquadscore
0,chr1,10101,10800,CTAACCCAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAAC...,TUF,63.0,50.0,84.0,chr1,10101.0,10800.0,2.02,0.0,0.48,0.88,True,CCCCAACCCCAACCCCAACCCCAACCCCAACCCCAA,36,CCCCTA,1,1.430615,[2.02]
2,chr1,11601,16900,CACCAGCAATGTCTAGGAATGCCTGTTTCTCCACAAAGTGTTTACT...,TUF,57.0,40.0,71.0,chr1,11601.0,16900.0,0.00,0.0,0.29,0.75,True,CCCCCAGCCCCCGGAGA,17,GGGGCC,0,0.000000,[]
41,chr1,80201,81000,ACATAAAGTGGGGAAAGGACACCCTATTCAACAAATGGTGCTGGTA...,TUF,40.0,30.0,57.0,chr1,80201.0,81000.0,0.00,0.0,0.28,0.60,True,,0,GGGGAA,0,0.000000,[]
43,chr1,81501,81900,ATACTTTATTTAGACATTTTGTTAGATTATTTTGACCAACTGAAGT...,TUF,29.0,26.0,36.0,chr1,81501.0,81900.0,0.00,0.0,0.23,0.39,True,,0,GGGGGTT,0,0.000000,[]
162,chr1,487601,488600,GAACAGGACTCATCTTCTGAGAAGGATGGACGCAGCCCCAACAAAT...,TUF,60.0,55.0,70.0,chr1,487601.0,488600.0,0.00,0.0,0.52,0.72,True,CCCCCAGGGTCCCCCGAT,18,CCCCAA,0,0.000000,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74220,chr9,137569600,137587499,AGCCAGCCAGCCACTCACCCACCATCCATCCATCCAACAATCCACC...,TUF,54.0,28.0,81.0,chr9,137569600.0,137587499.0,0.00,0.0,0.22,0.85,True,CCCCAGAACCCCCAAAAG,18,CCCCAC,0,0.000000,[]
74222,chr9,137587900,137590299,AAGTGAAACAAGCAAGCCAGAGGGACAAATGCCAGCCCTGTCCCTG...,TUF,60.0,37.0,87.0,chr9,137587900.0,137590299.0,0.00,0.0,0.36,0.91,True,GGGGCCGGGGCT,12,GGGGA,0,0.000000,[]
74224,chr9,137590600,137592099,CCGCCCCGCAACAACCAATAGGAGGAGGGCGAGTGCGCCGGCGCTC...,TUF,64.0,43.0,80.0,chr9,137590600.0,137592099.0,0.00,0.0,0.42,0.82,True,GGGGGAGGGGC,11,CCCCGC,0,0.000000,[]
74230,chr9,137603400,137604799,AGTATGCTTGGAGAAGAAGAGTCTAATGACAGGGCGGTTGGTGCTG...,TUF,57.0,41.0,73.0,chr9,137603400.0,137604799.0,0.00,0.0,0.33,0.74,True,GGGGGGCGGGGCA,13,GGGGCT,0,0.000000,[]


In [70]:
del tcdata['gquadscoremean']

In [158]:
tcdata['longest'],tcdata['length'] = zip(*tcdata.sequence.apply(lambda x: getlongestmatch(x,'4','','1','5','2','')))
#longest runtime here-only have to execute once
#took like 5-10mins

In [164]:
tcdata['match.group(0)'] = tcdata.sequence.apply(lambda x: shortrepeat(x,'4','','1','2','1',''))
#look at pickle instead of .copy() for this cell

In [64]:
tcdata['gquadcheck'],tcdata['gquadscore'] = zip(*tcdata.sequence.apply(lambda x: gquadcheck(x)))

In [ ]:
tcdata['gquadscoremax'] = tcdata.gquadscore.apply(lambda x: np.amax(x, initial=0))

In [27]:
tcdata['mingcs'],tcdata['maxgcs'],tcdata['acgt'] = zip(*tcdata.sequence.apply(lambda x: gcminmax(x, 100)))

In [155]:
tcdata['gquadscoremin'] = tcdata.gquadscore.apply(lambda x: np.amin(x, initial=0))

In [184]:
tcdata['gquadCount'] = tcdata.gquadscore.apply(lambda x: len(x))

In [185]:
#to find density of gquad per row
tcdata['gquadDensity per kb'] = tcdata['gquadCount']/tcdata['region_length'] * 1000

In [ ]:
#create row to show how many gquads in each row, by using len(array)
#show difference between maxscore and minscore between ISTATEs
#find density of gquad per row for each istate-divide count of gquad by region length and multiply by 100 or 1000 to find per 100 or 1000base pair (kb thing) 
#and then plot to compare but make column and add to dataframe ^
#look into statistics for any significant difference like T test
#remember t test to see sign diff between means

In [191]:
coredata.describe()

,start_idx,end_idx,gc,gc_min,gc_max,repstart,repend,hmm_state_id,has_gquad,has_repeats,n_repeats,region_length,gquadscoremax,length,mingcs,maxgcs,gquadscoremin,gquadCount,gquadDensity per kb
count,2.557000e+03,2.557000e+03,2557.000000,2557.000000,2557.000000,2.557000e+03,2.557000e+03,2557.0,2557.000000,2557.000000,2557.000000,2557.000000,2557.000000,2557.000000,2557.000000,2557.000000,2557.0,2557.000000,2557.000000
mean,5.988134e+07,5.988180e+07,59.086820,55.423543,62.979273,5.988134e+07,5.988180e+07,3.0,0.398514,0.501369,6.253813,462.511928,0.855006,9.069613,0.276261,0.852053,0.0,2.241689,5.204890
std,6.025533e+07,6.025533e+07,15.781082,16.192266,16.302427,6.025533e+07,6.025533e+07,0.0,0.489688,0.500096,11.486201,673.139255,1.056095,14.401324,0.175749,0.142003,0.0,5.450827,9.516380
min,1.040100e+04,1.050000e+04,0.000000,0.000000,0.000000,1.040100e+04,1.050000e+04,3.0,0.000000,0.000000,0.000000,99.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
25%,1.145690e+07,1.145720e+07,47.000000,41.000000,53.000000,1.145690e+07,1.145720e+07,3.0,0.000000,0.000000,0.000000,199.000000,0.000000,0.000000,0.100000,0.800000,0.0,0.000000,0.000000
50%,3.649670e+07,3.649740e+07,63.000000,59.000000,67.000000,3.649670e+07,3.649740e+07,3.0,0.000000,1.000000,2.000000,299.000000,0.000000,0.000000,0.300000,0.900000,0.0,0.000000,0.000000
75%,1.028019e+08,1.028021e+08,70.000000,67.000000,74.000000,1.028019e+08,1.028021e+08,3.0,1.000000,1.000000,8.000000,499.000000,2.080000,15.000000,0.400000,1.000000,0.0,2.000000,6.688963
max,2.473735e+08,2.473736e+08,100.000000,100.000000,100.000000,2.473735e+08,2.473736e+08,3.0,1.000000,1.000000,135.000000,12699.000000,2.990000,157.000000,1.000000,1.000000,0.0,93.000000,100.250627


In [192]:
tcdata.describe()

,start_idx,end_idx,gc,gc_min,gc_max,repstart,repend,hmm_state_id,has_gquad,has_repeats,n_repeats,region_length,gquadscoremax,gquadscoremin,mingcs,maxgcs,length,gquadCount,gquadDensity per kb
count,5.100000e+03,5.100000e+03,5100.000000,5100.000000,5100.000000,5.100000e+03,5.100000e+03,5100.0,5100.000000,5100.000000,5100.000000,5100.000000,5100.000000,5100.0,5100.000000,5100.000000,5100.000000,5100.000000,5100.000000
mean,6.288722e+07,6.289245e+07,52.366275,36.683529,65.313922,6.288722e+07,6.289245e+07,2.0,0.197059,0.478039,5.719412,5231.549020,0.398355,0.0,0.071902,0.915510,10.954510,0.592157,0.189962
std,6.195009e+07,6.195012e+07,11.310603,14.063058,14.152355,6.195009e+07,6.195012e+07,0.0,0.397816,0.499566,16.046067,6893.204285,0.805476,0.0,0.122361,0.124474,10.129011,1.871834,1.225267
min,1.000100e+04,1.040000e+04,0.000000,0.000000,1.000000,1.000100e+04,1.040000e+04,2.0,0.000000,0.000000,0.000000,99.000000,0.000000,0.0,0.000000,0.100000,0.000000,0.000000,0.000000
25%,7.480975e+06,7.481874e+06,45.000000,27.000000,57.000000,7.480975e+06,7.481874e+06,2.0,0.000000,0.000000,0.000000,499.000000,0.000000,0.0,0.000000,0.900000,0.000000,0.000000,0.000000
50%,4.431535e+07,4.432210e+07,54.000000,34.000000,69.000000,4.431535e+07,4.432210e+07,2.0,0.000000,0.000000,0.000000,2299.000000,0.000000,0.0,0.000000,1.000000,13.000000,0.000000,0.000000
75%,1.102209e+08,1.102264e+08,60.000000,44.250000,75.000000,1.102209e+08,1.102264e+08,2.0,0.000000,1.000000,6.000000,7399.000000,0.000000,0.0,0.100000,1.000000,18.000000,0.000000,0.000000
max,2.473736e+08,2.473744e+08,87.000000,87.000000,95.000000,2.473736e+08,2.473744e+08,2.0,1.000000,1.000000,595.000000,48499.000000,2.450000,0.0,0.700000,1.000000,102.000000,19.000000,40.404040
